# Spotify Scraping
Il seguente notebook illustra il processo di raccolta dei dati per il progetto. 
Partendo dalle tracce salvate dal mio account di Spotify, ho cercato di accumulare più artisti significativi possibili nei seguenti modi:
1. Ricorsivamente, richiedendo all'API artisti simili ad un artista.
2. Per ogni traccia pubblicata da un certo artista, venivano salvati tutti gli artisti che hanno collaborato con lui.

Per artisti significativi si indendono gli artisti con almeno 200 follower e appartenti ad almeno un genere italiano.

La raccolta dei dati è avvenuta in maniera frammentata e non in pochi blocco. Questo, a discapito dell'efficenza, ha permesso la collezione delle informazioni possibile anche senza dover eseguire tutto il codice da zero.

## Accesso a Spotify tramite API

In [2]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import json
import time

In [6]:
with open('cred.json') as cred_file:
    cred = json.load(cred_file)[1]

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=cred["client_id"],
                                               client_secret=cred["client_secret"],
                                               redirect_uri=cred["redirect_uri"],
                                               scope=cred["scope"],
                                               username=cred["username"]))

## Salvataggio di tutti gli artisti de "La mia Libreria"

In [21]:
artisti = []
limit = 50
after = 0
results = sp.current_user_saved_tracks(limit=limit, offset=after)

while True:
    results = sp.current_user_saved_tracks(limit=limit, offset=after)
    print("Limite: ", limit, ", Offset: ", after, ", Numero di risultati: ", len(results['items']))

    for item in results['items']:
        track = item["track"]
        if track["type"] == "track":
            for artist in track["artists"]:
                if not list(filter(lambda x: x["uri"] == artist["uri"], artisti)):
                    artisti.append({"uri":  artist["uri"], "name": artist["name"]})
    
    after += limit
    if len(results['items']) < limit:   
        break

Limite:  50 , Offset:  0 , Numero di risultati:  50
Limite:  50 , Offset:  50 , Numero di risultati:  50
Limite:  50 , Offset:  100 , Numero di risultati:  50
Limite:  50 , Offset:  150 , Numero di risultati:  50
Limite:  50 , Offset:  200 , Numero di risultati:  50
Limite:  50 , Offset:  250 , Numero di risultati:  50
Limite:  50 , Offset:  300 , Numero di risultati:  50
Limite:  50 , Offset:  350 , Numero di risultati:  50
Limite:  50 , Offset:  400 , Numero di risultati:  50
Limite:  50 , Offset:  450 , Numero di risultati:  50
Limite:  50 , Offset:  500 , Numero di risultati:  27


Dopo aver accumulato tutti gli artisti della libreria dell'account Spotify, è stato fatto un primo filtraggio, eliminando tutti gli artisti non appartenenti ad almeno un genere italiano.

In [22]:
def divide_chunks(l, n):
    for i in range(0, len(l), n): 
        yield l[i:i + n]


def artist_filter(genre, artists):
    results = []
    artists_id = [artist["uri"] for artist in artists]


    for artists_id_chunk in divide_chunks(artists_id, 50):
        artists_more_info = sp.artists(artists_id_chunk)

        for artist in artists_more_info["artists"]:
            for music_genre in artist["genres"]:
                if genre in music_genre:
                    result = {}
                    result["uri"], result["name"], result["followers"] = artist["uri"], artist["name"], artist["followers"]["total"]
                    results.append(result)
                    break

    return results


# Filtro per artisti italiani
artisti = artist_filter("italian", artisti)

# Salvataggio dei progressi fatti
with open('artisti_liberia.json', 'w') as outfile:
    json.dump(artisti, outfile, indent=4)

KeyboardInterrupt: 

In [ ]:
# Checkpoint 1
with open('artisti_liberia.json') as json_file:
    artisti = json.load(json_file)

## Identificazione di artisti simili tra di loro
Tramite la funzione `sp.artist_related_artists(artista_id)`, Spotify ritorna una serie di artisti simili a quello passato per parametro. Utilizzando questa funzione ricorsivamente, si accumulano ulteriori artisti italiani.

In [ ]:
def flatten(t):
    return [item for sublist in t for item in sublist]

def get_artists_from_artists(artist_uri, missing_steps):
    if missing_steps > 0:
        try:
            artisti = artist_filter("italian", sp.artist_related_artists(artist_uri)["artists"])
        except Exception as e:
            print(e)
            time.sleep(5)
            artisti = artist_filter("italian", sp.artist_related_artists(artist_uri)["artists"])

        partial_result = flatten([get_artists_from_artists(artista["uri"], missing_steps - 1) for artista in artisti])
    
        return artisti + partial_result
    else:
        try:
            return artist_filter("italian", sp.artist_related_artists(artist_uri)["artists"])
        except Exception as e:
            print(e)
            time.sleep(5)
            return artist_filter("italian", sp.artist_related_artists(artist_uri)["artists"])


recursive_artist = get_artists_from_artists(artisti[0]["uri"], 2)

# Eliminazione degli artisti che risulano duplicati nella lista
recursive_artist_no_dupl = []
for x in recursive_artist:
    if x not in recursive_artist_no_dupl:
        recursive_artist_no_dupl.append(x)

with open('artisti_ricorsivi.json', 'w') as json_file:
    json.dump(recursive_artist_no_dupl, json_file, indent=4)
    
len(recursive_artist_no_dupl)

In [ ]:
# Checkpoint 2
with open('artisti_ricorsivi.json', 'r') as json_file:
    recursive_artists = json.load(json_file)

## Collezione di artisti basata sulle collaborazioni

In [ ]:
artists_full = {}

for counter, artist in enumerate(recursive_artists):
    results = sp.artist_albums(artist["uri"], album_type='album')
    albums = results['items']
    
    while results['next']:
        results = sp.next(results)
        albums.extend(results['items'])

    for album in albums:
        album_tracks = sp.album_tracks(album["id"], limit=50)
        for track in album_tracks["items"]:
            for track_artist in track["artists"]:
                artists_full[track_artist["name"]] = {
                    "id": track_artist["id"]
                }
    print(f"{counter}/{len(recursive_artists)} artisti caricati")
    time.sleep(3)

print(artists_full)

In [ ]:
len(artists_full)
with open('artisti_full.json', 'w') as json_file:
    json.dump(artists_full, json_file, indent=4)

Teniamo solo gli artisti con almeno 200 follower e filtriamo solo per artisti italiani

## Filtraggio dei risultati
A questo punto gli artisti con meno di 200 followers e che non appartengono ad almeno un genere italiani vengono scartati.

In [ ]:
artisti_filtrati = {}

for artist_name, artist_id in artists_full.items():
    artist_data = sp.artist(f"spotify:artist:{artist_id['id']}")
    
    for music_genre in artist_data["genres"]:
        if "italian" in music_genre:
            if artist_data["followers"]["total"] > 200:
                artisti_filtrati[artist_name] = artist_id
                artisti_filtrati[artist_name]["followers"] = artist_data["followers"]["total"]
                break

In [51]:
with open('artisti_filtrati.json', 'r') as json_file:
    artisti_filtrati = json.load(json_file)

## Scraper album/tracks
A questo punto, per ogni artista, vengono salvati tutti le canzoni e gli album pubblicati

In [54]:
artists_albums_tracks = {}

for i, artist_info in enumerate(artisti_filtrati.items()):
    artist, info = artist_info
    print(f"Artista {i+1} su {len(artisti_filtrati)}")
    albums = sp.artist_albums(f"spotify:artist:{info['id']}")

    albums_to_add = {}
    for album in albums["items"]:
        albums_to_add[album["id"]] = {
            "name": album["name"], 
            "release_date": album["release_date"], 
            "total_tracks": album["total_tracks"],
        }

        tracks_to_add = {}
        for track in sp.album_tracks(album["id"], limit=50)["items"]:
            tracks_to_add[track["id"]] = {
                "name": track["name"], 
                "duration_ms": track["duration_ms"], 
                "explicit": track["explicit"],
                "artists": [track_artist["id"] for track_artist in track["artists"]]
            }
        
        albums_to_add[album["id"]]["tracks"] = tracks_to_add
        artists_albums_tracks[info["id"]] = {
            "name": artist,
            "followers": info["followers"],
            "albums": albums_to_add,
        }
    
    time.sleep(0.5)


Artista 1 su 478
Artista 2 su 478
Artista 3 su 478
Artista 4 su 478
Artista 5 su 478
Artista 6 su 478
Artista 7 su 478
Artista 8 su 478
Artista 9 su 478
Artista 10 su 478
Artista 11 su 478
Artista 12 su 478
Artista 13 su 478
Artista 14 su 478
Artista 15 su 478
Artista 16 su 478
Artista 17 su 478
Artista 18 su 478
Artista 19 su 478
Artista 20 su 478
Artista 21 su 478
Artista 22 su 478
Artista 23 su 478
Artista 24 su 478
Artista 25 su 478
Artista 26 su 478
Artista 27 su 478
Artista 28 su 478
Artista 29 su 478
Artista 30 su 478
Artista 31 su 478
Artista 32 su 478
Artista 33 su 478
Artista 34 su 478
Artista 35 su 478
Artista 36 su 478
Artista 37 su 478
Artista 38 su 478
Artista 39 su 478
Artista 40 su 478
Artista 41 su 478
Artista 42 su 478
Artista 43 su 478
Artista 44 su 478
Artista 45 su 478
Artista 46 su 478
Artista 47 su 478
Artista 48 su 478
Artista 49 su 478
Artista 50 su 478
Artista 51 su 478
Artista 52 su 478
Artista 53 su 478
Artista 54 su 478
Artista 55 su 478
Artista 56 su 478
A

In [55]:
with open('artists_albums_tracks.json', 'w') as json_file:
    json.dump(artists_albums_tracks, json_file, indent=4)

## Passaggio a Dataframe

In [3]:
with open('artists_albums_tracks.json', 'r') as json_file:
    artists_albums_tracks = json.load(json_file)

In [104]:
import pandas as pd

every_artist = {"id": [], "name": [], "followers": [], "album": []}
every_album = {
    "album_id": [], 
    "album_name": [], 
    "release_date": [], 
    "total_tracks": [], 
    "track_id": [], 
    "track_name": [], 
    "duration_ms": [],
    "explicit": [], 
    "artists": [],
    "lyrics": []
}
same_albumname, same_tracks, same_artist = False, False, False

for artist, artist_info in artists_albums_tracks.items():
    for album_id in list(artist_info["albums"].keys()):
        if album_id not in every_artist["album"]:
            every_artist["id"].append(artist)
            every_artist["name"].append(artist_info["name"])
            every_artist["followers"].append(artist_info["followers"])
            every_artist["album"].append(album_id)
    
    for album, album_info in artist_info["albums"].items():
        if album not in every_album["album_id"] and "Live" not in album_info["name"] and album_info["total_tracks"] > 1:
            for track, track_info in album_info["tracks"].items():
                if "Live" not in track_info["name"]: 
                    every_album["album_id"].append(album)
                    every_album["album_name"].append(album_info["name"])
                    every_album["release_date"].append(album_info["release_date"])
                    every_album["total_tracks"].append(album_info["total_tracks"])
                    every_album["track_id"].append(track)
                    every_album["track_name"].append(track_info["name"])
                    every_album["duration_ms"].append(track_info["duration_ms"])
                    every_album["explicit"].append(track_info["explicit"])
                    every_album["artists"].append(track_info["artists"])
                    every_album["lyrics"].append(list())

artists = pd.DataFrame.from_dict(every_artist)
albums = pd.DataFrame.from_dict(every_album)
#tracks = pd.DataFrame.from_dict(every_track)
albums #49579   

,album_id,album_name,release_date,total_tracks,track_id,track_name,duration_ms,explicit,artists,lyrics
0,7DvK67C21i6go7olhjvLgT,FLOP,2021-10-01,17,5mJDKaEsF02C8BOZoJ14rl,ANTIPATICO,117240,True,[3hBQ4zniNdQf1cqqo6hzuW],[]
1,7DvK67C21i6go7olhjvLgT,FLOP,2021-10-01,17,2v4kWcQ7LWML8TEQmLDAEO,MI SENTO BENE,183089,True,[3hBQ4zniNdQf1cqqo6hzuW],[]
2,7DvK67C21i6go7olhjvLgT,FLOP,2021-10-01,17,6FCWM1nmcM5fEPgS2EUAtB,CRIMINALE,158527,True,[3hBQ4zniNdQf1cqqo6hzuW],[]
3,7DvK67C21i6go7olhjvLgT,FLOP,2021-10-01,17,01BCvCKQNDmFQQQSjrzPnm,GHIGLIOTTINA - feat. Noyz Narcos,186733,True,"[3hBQ4zniNdQf1cqqo6hzuW, 49UAapOfpOg1ZOU4xf2NgY]",[]
4,7DvK67C21i6go7olhjvLgT,FLOP,2021-10-01,17,25zlvKhD4wtNyjWQ91APzB,IN TRAPPOLA,112867,True,[3hBQ4zniNdQf1cqqo6hzuW],[]
...,...,...,...,...,...,...,...,...,...,...
48683,3OKkIqlgQos2kEmtxGxchG,"Street Jazz (Hip Hop Jazz, Electro Jazz, Nu Ja...",2017-08-07,25,65lZbnTDx14JaQtQALF33G,Psycho Bop,229333,False,[033fOLZg0lEdp1exVyTCMt],[]
48684,3OKkIqlgQos2kEmtxGxchG,"Street Jazz (Hip Hop Jazz, Electro Jazz, Nu Ja...",2017-08-07,25,6FsQn2vC3JFXDeS6f5wet9,Spaceship Lounge,229346,False,[3O0Mg4SG77p7gJa3v7Jjm0],[]
48685,3OKkIqlgQos2kEmtxGxchG,"Street Jazz (Hip Hop Jazz, Electro Jazz, Nu Ja...",2017-08-07,25,1BHMqi4eLuofXiFuKtbAWw,It's Monday too,265372,False,[0ZweNRF5ce7z4L8Df6gqig],[]
48686,3OKkIqlgQos2kEmtxGxchG,"Street Jazz (Hip Hop Jazz, Electro Jazz, Nu Ja...",2017-08-07,25,6ZWPPDLtmezASVDBirQ445,Everlasting Rose - Belladonna Remix,346060,False,"[3pNXwV8bJgtisiTeRYqmdh, 0tOi96DSOq8s9Qadc6CxU6]",[]


In [105]:
print(len(albums))

for track in albums.itertuples():
    albums.at[track.Index, 'artists'] = " ".join(track.artists)

albums = albums.drop_duplicates(subset=['album_name', 'track_name', 'duration_ms'], keep='first')
albums_no_dups = albums.drop_duplicates(subset=['track_name', 'duration_ms', 'artists'], keep='last')

for track in albums_no_dups.itertuples():
    albums_no_dups.at[track.Index, 'artists'] = track.artists.split(" ")
    
for track in albums.itertuples():
    albums.at[track.Index, 'artists'] = track.artists.split(" ")

print("Album trash away", len(albums), "Albums no dups", len(albums_no_dups))
print(albums[albums.track_name == "Bruce Willis"])
print(albums_no_dups[albums_no_dups.track_name == "Bruce Willis"])

48688
Album trash away 46493 Albums no dups 42028
                   album_id                 album_name release_date  \
324  4xzpCepiaf3oQaywSGqcGV  Status (Vendetta Edition)   2016-01-22   
355  579t0HatXMKQnAfQ9FSfJd                     Status   2015-02-10   

     total_tracks                track_id    track_name  duration_ms  \
324            53  6w9fEIuKPFDt99Z5hzxoyj  Bruce Willis       130546   
355            20  11WeKVrdKIm16BjmvH2QEb  Bruce Willis       130546   

     explicit                   artists lyrics  
324      True  [5AZuEF0feCXMkUCwQiQlW7]     []  
355      True  [5AZuEF0feCXMkUCwQiQlW7]     []  
                   album_id album_name release_date  total_tracks  \
355  579t0HatXMKQnAfQ9FSfJd     Status   2015-02-10            20   

                   track_id    track_name  duration_ms  explicit  \
355  11WeKVrdKIm16BjmvH2QEb  Bruce Willis       130546      True   

                      artists lyrics  
355  [5AZuEF0feCXMkUCwQiQlW7]     []  


## Scraper canzoni
Tramite A-Z Lyrics si scaricano i testi di ogni canzone collezionata precendetemente.

In [263]:
import azapi
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from proxy import ProxyManager
import re
import unicodedata

def remove_accents(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    only_ascii = nfkd_form.encode('ASCII', 'ignore')
    return only_ascii.decode("utf-8") 

def get_clean_lyrics(text):
    text_tokens = word_tokenize(text)
    tokens_without_sw = [word.lower() for word in text_tokens if not word in stopwords.words("italian")]
    clean_tokens = [word for word in tokens_without_sw if re.findall(r"[^-!$%^&*()_+|~=`{}\[\]:\";'<>?,.\/]", word)]
    return " ".join(sorted(set(clean_tokens), key=clean_tokens.index))

tracks_final = pd.DataFrame()
gestore_proxy = ProxyManager("royal.txt", 1)

for track in albums_no_dups.itertuples():
    track_name = track.track_name.lower().replace("(", "").replace(")", "").replace(" ", "").replace("-","")
    track_name = track_name.split("feat.")[0]
    track_name = track_name.split("prod.")[0]
    track_name = track_name.split("skit")[0]
    #track_name = "".join(re.findall(r"[a-z0-9]+", track_name))
    track_name = remove_accents(track_name)
    track_name = "".join(re.findall(r"[a-z0-9]+", track_name))

    API = azapi.AZlyrics()

    try:
        API.artist = artists[artists.album == track.album_id].name.iloc[0]

        if API.artist == "Guè":
            API.artist = "guepequeno"
    except KeyError as e:
        print("La canzone è probabilmente un singolo", e)
        print(track.album_id, track.track_name, API.artist, track_name)
    else:
        API.title = track_name

        if not track.lyrics:
            print(API.artist, API.title, "in download")
            proxy = gestore_proxy.pick_proxy()

            lyrics = API.getLyrics(sleep=15)

            if isinstance(lyrics, str):
                albums_no_dups.at[track.Index, 'lyrics'] = get_clean_lyrics(lyrics)
            else:
                print(lyrics, "Errore")
                if lyrics == 1:
                    albums_no_dups.at[track.Index, 'lyrics'] = "NA"
                    continue
                else:
                    break
                
        else:
            print(track.track_name, ": già scaricato")

ANTIPATICO : già scaricato
MI SENTO BENE : già scaricato
CRIMINALE : già scaricato
GHIGLIOTTINA - feat. Noyz Narcos : già scaricato
IN TRAPPOLA : già scaricato
LA CHIAVE - feat. Marracash : già scaricato
KUMITE : già scaricato
CHE NE SO : già scaricato
YHWH - feat. Guè : già scaricato
HELLVISBACK 2 : già scaricato
A DIO : già scaricato
FUORI DI TESTA - feat. Par-T-One : già scaricato
MARLA : già scaricato
L'ANGELO CADUTO - feat. Shari : già scaricato
VIVO : già scaricato
FLOP! : già scaricato
ALDO RITMO : già scaricato
90MIN : già scaricato
STAI ZITTO (feat. Fabri Fibra) : già scaricato
RICCHI E MORTI : già scaricato
DISPOVERY CHANNEL (feat. Nitro) : già scaricato
CABRIOLET (feat. Sfera Ebbasta) : già scaricato
HO PAURA DI USCIRE : già scaricato
SPARARE ALLA LUNA (feat. Coez) : già scaricato
PXM : già scaricato
IL CIELO NELLA STANZA (feat. NSTASIA) : già scaricato
TIE' : già scaricato
ORA CHE FAI? : già scaricato
PERDONAMI : già scaricato
LUNEDI' : già scaricato
Don Medellín (feat. Ros

KeyboardInterrupt: 

In [264]:
albums_no_dups.to_pickle("albums_no_dups3.pkl")

In [262]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

albums_no_dups[albums_no_dups.index > 3440]

#for a in albums_no_dups[albums_no_dups.index > 1088].itertuples():
#    albums_no_dups.at[a.Index, 'lyrics'] = []


,album_id,album_name,release_date,total_tracks,track_id,track_name,duration_ms,explicit,artists,lyrics
3441,7GK5xFWfFQBUisVtld1PiX,Beat Coin,2021-09-03,11,504QJc7vkwG4vuV3abgASn,Sembra un film,187415,False,"[4f4ERDE3xkOakb8zOnlaRp, 5dXlc7MnpaTeUIsHLVe3n...",[]
3442,7GK5xFWfFQBUisVtld1PiX,Beat Coin,2021-09-03,11,4BT0Q5cqw8cUc5KiBtV19W,Mille colori,209102,False,"[4f4ERDE3xkOakb8zOnlaRp, 05kigUQCchmbJDmhDiYiS...",[]
3443,7GK5xFWfFQBUisVtld1PiX,Beat Coin,2021-09-03,11,3KSWdhvSUpxodLiZGTVvxd,Un'altra brasca,206117,True,"[4f4ERDE3xkOakb8zOnlaRp, 3GNq6JwYWh8B2xujy6ub0...",[]
3444,7GK5xFWfFQBUisVtld1PiX,Beat Coin,2021-09-03,11,1eEcNZt66gX1tCX3rdYZ2p,"Wild Boys, Pt. 2",242307,True,"[4f4ERDE3xkOakb8zOnlaRp, 6yqDtnAXl4uu8ka0gTiUn...",[]
3445,7GK5xFWfFQBUisVtld1PiX,Beat Coin,2021-09-03,11,3L7sUwVUiVAn8miWF3DVsq,Stress,238000,True,"[4f4ERDE3xkOakb8zOnlaRp, 0ba6wsfB7G2rhdHMebCdO3]",[]
...,...,...,...,...,...,...,...,...,...,...
48683,3OKkIqlgQos2kEmtxGxchG,"Street Jazz (Hip Hop Jazz, Electro Jazz, Nu Ja...",2017-08-07,25,65lZbnTDx14JaQtQALF33G,Psycho Bop,229333,False,[033fOLZg0lEdp1exVyTCMt],[]
48684,3OKkIqlgQos2kEmtxGxchG,"Street Jazz (Hip Hop Jazz, Electro Jazz, Nu Ja...",2017-08-07,25,6FsQn2vC3JFXDeS6f5wet9,Spaceship Lounge,229346,False,[3O0Mg4SG77p7gJa3v7Jjm0],[]
48685,3OKkIqlgQos2kEmtxGxchG,"Street Jazz (Hip Hop Jazz, Electro Jazz, Nu Ja...",2017-08-07,25,1BHMqi4eLuofXiFuKtbAWw,It's Monday too,265372,False,[0ZweNRF5ce7z4L8Df6gqig],[]
48686,3OKkIqlgQos2kEmtxGxchG,"Street Jazz (Hip Hop Jazz, Electro Jazz, Nu Ja...",2017-08-07,25,6ZWPPDLtmezASVDBirQ445,Everlasting Rose - Belladonna Remix,346060,False,"[3pNXwV8bJgtisiTeRYqmdh, 0tOi96DSOq8s9Qadc6CxU6]",[]


In [141]:
for track in albums_no_dups.itertuples():
    if track.album_id != "6oPdp8t50DQ9fvYIUm43uW":
        if track.lyrics == []:
            albums_no_dups.at[track.Index, 'lyrics'] = "NA"
    else:
        break